In [1]:
#import libraries
import pandas as pd

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
a = pd.read_csv('Answers.csv', usecols = ['Id', 'OwnerUserId', 'CreationDate', 'ParentId', 'Score', 'Body'], low_memory=False)
q = pd.read_csv('Questions.csv', usecols = ['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title', 'Body'], low_memory=False)
t = pd.read_csv('Tags.csv', usecols = ['Id', 'Tag'], low_memory=False)

q_clean = q
q_clean = q_clean[q_clean['Id'].str.contains('\D') == False]
q_clean['Id'] = q_clean['Id'].astype(str).astype(int)
q_clean = q_clean[q_clean['Score'].str.contains('\D') == False]
q_clean['Score'] = q_clean['Score'].astype(str).astype(int)
q_clean.drop(["OwnerUserId"], axis=1, inplace=True)
q_clean['Body'] = q_clean['Body'].astype(str)

a_clean = a
a_clean.drop(["Id", "OwnerUserId"], axis=1, inplace=True)
a_clean.rename(columns={'ParentId' : 'Id'}, inplace=True)

In [3]:
import re

q_clean['Title'] = \
q_clean['Title'].map(lambda x: re.sub('[,\.!?]', '', x))
q_clean['Title'] = \
q_clean['Title'].map(lambda x: x.lower())

In [4]:
q_clean['Body'] = \
q_clean['Body'].map(lambda x: re.sub('[,\.!?]', '', x))
q_clean['Body'] = \
q_clean['Body'].map(lambda x: x.lower())

In [5]:
a_clean['Body'] = \
a_clean['Body'].map(lambda x: re.sub('[,\.!?]', '', x))
a_clean['Body'] = \
a_clean['Body'].map(lambda x: x.lower())

In [6]:
print(t.shape) #1,048,575 rows
print(t['Tag'].unique().shape) # 21,984 unique values
#
unique_tags = t['Tag'].value_counts(ascending=False).reset_index()
# tags = pd.DataFrame(t[t.Tag == '*c*'])
# print(tags.shape)
# tags.head(15)

(1048575, 2)
(21984,)


<h1>Get all Id from posts with relevent tags</h1>

attempt 1:
only gets id's from posts tagged with the language. no information on libraries. 

In [7]:
t_java = pd.DataFrame(t[t.Tag == 'java'])
t_python = pd.DataFrame(t[t.Tag == 'python'])
t_cpp = pd.DataFrame(t[t.Tag == 'c++'])
t_java = pd.DataFrame(t[t.Tag == 'java'])
t_python = pd.DataFrame(t[t.Tag == 'python'])
t_cpp = pd.DataFrame(t[t.Tag == 'c++'])

attempt 2:
gets id's from posts tagged with language or libraries. case insensitive, gets more cases where the language is hidden inside a larger string. slow and long imnplementation. doesn't separate by language. 

In [8]:
out1 = t[t['Tag'].astype(str).str.contains('java', case=False)]
out2 = t[t['Tag'].astype(str).str.contains('python', case=False)]
out3 = t[t['Tag'].astype(str).str.contains('c\+\+', case=False)]
out4 = t[t['Tag'].astype(str).str.contains('cpp', case=False)]
#t_cpp = pd.DataFrame(t[t.Tag == 'c++'])
frames = [out1, out2, out3, out4]
out = pd.concat(frames)
out.tail()

,Id,Tag
902207,11259570,cppcheck
962338,11916290,rcpp
1017367,12528930,activemq-cpp
1032255,12691710,rcpp
1047083,12857860,rcpp


attempt 3: speed and clean up implementation. separate by language

In [9]:
cpp_tags_list = ['c\+\+', 'cpp', 'boost', 'poco', 'googletest', 'tbb', 'cinder', 'wtl',
                 'stdvector', 'stdstring', 'stdlist']
c_tags_cpp_compatible = ['ffmpeg', 'openssl', 'sqlite', 'opencv']
java_tags_list = ['java', 'guava', 'cglib', 'apache', 'jackson', 'c3p0', 'jms',
                 'jodatime', 'gson', 'log4j', 'hslf', 'slf4j', 'junit', 
                 'mockito', 'spring', 'jmx', 'jaxb', 'httpclient'] #remember to remove Javascript!! 
python_tags_list = ['python', 'django', 'pybrain', 'keras', 'lightgbm', 'eli5', 'theano', 
                    'scipy', 'scrapy', 'scikit-learn', 'scikit', 'pygame', 'matplotlib', 'pytorch',
                    'pandas', 'numpy']
#libraries that encompass multiple languages
multiple_tags_list = ['eclipse', 'wpf', 'tensorflow', 'maven', 'xerces'] 
# eclipse is java and c, sometimes c++
# tensorflow is python and c++ and javascript
# maven is java and c++
# xerces is java and c++
#std can appear in many languages????
# WPF is C and C++


cpp_pattern = '|'.join(cpp_tags_list)
java_pattern = '|'.join(java_tags_list)
python_pattern = '|'.join(python_tags_list)

cpp_ids = t[t['Tag'].astype(str).str.contains(cpp_pattern, case=False)]
java_ids = t[t['Tag'].astype(str).str.contains(java_pattern, case=False)]
python_ids = t[t['Tag'].astype(str).str.contains(python_pattern, case=False)]

#remove .h from c++, remove petapoco
cpp_remove_list = ['', '']
#remove Javascript, remove json, remove log4js




In [10]:

print(cpp_ids['Tag'].value_counts(ascending=False).reset_index())
print(java_ids['Tag'].value_counts(ascending=False).reset_index())
print(python_ids['Tag'].value_counts(ascending=False).reset_index())

                 Tag  count
0                c++  14691
1         visual-c++   1012
2              boost    731
3              c++11    460
4            c++-cli    230
..               ...    ...
110            c++1z      1
111         yaml-cpp      1
112     c++-concepts      1
113  c++builder-2009      1
114       boost-move      1

[115 rows x 2 columns]
               Tag  count
0             java  29266
1       javascript  26199
2           spring   1954
3           apache   1770
4          java-ee    979
..             ...    ...
245    openid4java      1
246        javapos      1
247    java-canvas      1
248  spring-nature      1
249      spring-js      1

[250 rows x 2 columns]
                     Tag  count
0                 python  13470
1                 django   3862
2          django-models    522
3                  numpy    343
4             python-3.x    315
..                   ...    ...
194    ironpython-studio      1
195    python-webbrowser      1
196  django-ajax

In [11]:
questions_java = pd.merge(java_ids, q_clean, how='inner', on=['Id'])
answers_java = pd.merge(java_ids, a_clean, how='inner', on=['Id'])

questions_python = pd.merge(python_ids, q_clean, how='inner', on=['Id'])
answers_python = pd.merge(python_ids, a_clean, how='inner', on=['Id'])

questions_cpp = pd.merge(cpp_ids, q_clean, how='inner', on=['Id'])
answers_cpp = pd.merge(cpp_ids, a_clean, how='inner', on=['Id'])

In [12]:
print(questions_java)

             Id                      Tag          CreationDate  \
0          4080                     java  2008-08-06T22:45:27Z   
1          4630                     java  2008-08-07T12:26:50Z   
2          4630           java-web-start  2008-08-07T12:26:50Z   
3          7540               javascript  2008-08-11T05:24:33Z   
4          7720                     java  2008-08-11T12:37:50Z   
...         ...                      ...                   ...   
64918  12873840               javascript  2012-10-13T14:03:28Z   
64919  12873910  facebook-javascript-sdk  2012-10-13T14:11:57Z   
64920  12873970                     java  2012-10-13T14:19:01Z   
64921  12873990               javascript  2012-10-13T14:22:03Z   
64922  12874050                     java  2012-10-13T14:28:55Z   

                 ClosedDate  Score  \
0      2012-05-05T13:39:59Z     92   
1                       NaN      8   
2                       NaN      8   
3                       NaN      6   
4                

In [13]:
print(answers_java)

              Id                      Tag          CreationDate  Score  \
0           4080                     java  2008-08-07T00:20:41Z      1   
1           4080                     java  2008-08-07T07:18:26Z      4   
2           4080                     java  2008-08-07T07:21:49Z      3   
3           4080                     java  2008-08-08T14:53:12Z      0   
4           4080                     java  2008-08-16T18:48:43Z     11   
...          ...                      ...                   ...    ...   
146594  12873910  facebook-javascript-sdk  2012-10-13T18:47:20Z      9   
146595  12873970                     java  2012-10-13T14:24:35Z      0   
146596  12873970                     java  2012-10-13T17:33:34Z      2   
146597  12873990               javascript  2012-10-13T14:22:52Z      1   
146598  12874050                     java  2012-10-13T18:50:36Z      1   

                                                     Body  
0       <p>we use findbugs and jdepend integrated w

In [14]:
questions_java[['Body','Title']].to_csv("Questions_Java.csv", header=None, index=None, sep=' ')
answers_java[['Body']].to_csv("Answers_Java.csv", header=None, index=None, sep=' ')
questions_cpp[['Body','Title']].to_csv("Questions_Cpp.csv", header=None, index=None, sep=' ')
answers_cpp[['Body']].to_csv("Answers_Cpp.csv", header=None, index=None, sep=' ')
questions_python[['Body','Title']].to_csv("Questions_Python.csv", header=None, index=None, sep=' ')
answers_python[['Body']].to_csv("Answers_Python.csv", header=None, index=None, sep=' ')

In [ ]:
https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

In [19]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'code', 'error', 'li'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = questions_java['Body']
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sewol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['analysis', 'tools', 'java', 'projects', 'interested', 'kinds', 'ul', 'static', 'analysis', 'tools', 'findbugs', 'pmd', 'others', 'coverage', 'tools', 'cobertura', 'emma', 'others', 'instrumentation', 'based', 'tools', 'anything', 'else', 'missing', 'something', 'ul', 'applicable', 'also', 'state', 'build']


In [17]:
print(data_words)

[['body'], ['title']]


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

In [ ]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]